# stockdata

> This module does the following: 1) Pull latest data from NSE 2) Calculate various technical indicators 3) Save the updated, generated data.

In [ ]:
#| default_exp stockdata

In [ ]:
#| export
import pandas as pd
from datetime import datetime
import requests
import zipfile
import io
import time
import random
from pathlib import Path
import nbdev

In [ ]:
#| export
base_path = nbdev.config.get_config().lib_path

In [ ]:
#| export
def _fetch_daily_bhavcopy(year, month, day, random_delay=False):

    nse_url = "https://archives.nseindia.com/content/historical/EQUITIES"
    nse_url_2 = "https://www1.nseindia.com/content/historical/EQUITIES"

    user_agents_list = [
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]

    """Fetches bhavcopy data for a given date, unzips and saves to local path"""
    # Define paths
    file_name = f"cm{day:02}{month}{year}bhav.csv.zip"
    file_url = f"{nse_url}/{year}/{month}/{file_name}"
    file_url_2 = f"{nse_url_2}/{year}/{month}/{file_name}"
    zip_path = base_path / f"../Data/Bhavcopy/Zips/cm{day:02}{month}{year}bhav.csv.zip"
    unzip_dir = base_path / "../Data/Bhavcopy/Raw"

    # If file exists, unzip and save to local path
    if Path(zip_path).is_file():
        print(f"File {file_name} already exists.. unzipping")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_dir)
    else:
        if random_delay:
            time.sleep(random.randint(1, 10))

        headers = headers={'User-Agent': random.choice(user_agents_list)}
        print(f"Downloading:{file_url}")

        with requests.Session() as s:
            r = s.get(file_url, allow_redirects=False, headers=headers)

            if r.status_code != 200:    
                print(f"Status: {r.status_code}: {file_url}")
                r = s.get(file_url_2, allow_redirects=False, headers=headers)

            if r.status_code != 200:
                print(f"Status: {r.status_code}: {file_url_2}")
                print(f"~~~~~~~~~~~~> Error downloading file: {file_name}")
                # raise Exception(f"Error downloading file: {file_name}")
            else:
                open(zip_path, "wb").write(r.content)
                with zipfile.ZipFile(io.BytesIO(r.content)) as zip_ref:
                    zip_ref.extractall(unzip_dir)
                    print(f"Processed file: {file_name}")

In [ ]:
#| export

def fetch_bhavcopy_data_for_range(start_date, end_date):
    """Fetches bhavcopy data for a given date range and returns a pandas dataframe"""
    for d in pd.date_range(start_date, end_date):
        # Get Year, Month, Day
        year = d.year
        month = d.strftime("%B").upper()[:3]
        day = d.date().strftime("%d")
        
        # If weekday, process file
        if 0 <= d.weekday() < 5:
            _fetch_daily_bhavcopy(year, month, day, random_delay=True)

In [ ]:
# Define date range
start_date = datetime(2023, 2, 11)
end_date = datetime.now()

# Fetch data from NSE
fetch_bhavcopy_data_for_range(start_date, end_date)

File cm01FEB2023bhav.csv.zip already exists.. unzipping
File cm02FEB2023bhav.csv.zip already exists.. unzipping
File cm03FEB2023bhav.csv.zip already exists.. unzipping
Downloading:https://archives.nseindia.com/content/historical/EQUITIES/2023/FEB/cm06FEB2023bhav.csv.zip
Processed file: cm06FEB2023bhav.csv.zip
Downloading:https://archives.nseindia.com/content/historical/EQUITIES/2023/FEB/cm07FEB2023bhav.csv.zip
Processed file: cm07FEB2023bhav.csv.zip
Downloading:https://archives.nseindia.com/content/historical/EQUITIES/2023/FEB/cm08FEB2023bhav.csv.zip
Processed file: cm08FEB2023bhav.csv.zip
Downloading:https://archives.nseindia.com/content/historical/EQUITIES/2023/FEB/cm09FEB2023bhav.csv.zip
Processed file: cm09FEB2023bhav.csv.zip
Downloading:https://archives.nseindia.com/content/historical/EQUITIES/2023/FEB/cm10FEB2023bhav.csv.zip
Processed file: cm10FEB2023bhav.csv.zip


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()